In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Ruta Ángel

df = pd.read_csv('/content/drive/MyDrive/CEU SAN PABLO/TFM/datos_imagenes.csv')

image_folder_path = '/content/drive/MyDrive/CEU SAN PABLO/TFM/fotos'

In [2]:
# Ruta Álvaro

df = pd.read_csv('/content/drive/MyDrive/tfm/TFM/datos_imagenes.csv')

image_folder_path = '/content/drive/MyDrive/tfg/TFM/fotos'

In [3]:
df.head()

,address,agency_name,bathroom_count,bedroom_count,floor,latitude,longitude,lot_size,property_description,property_images,...,neighborhood,exterior,ascensor,dist_metro_m,property_type_flat,energy_certificate_encoded,property_type_chalet,altura_techo,tipo_suelo,estilo
0,Calle de Zurbano,Ambassador,4,5,5.0,40.435694,-3.691642,344,Fantástico tríplex muy luminoso y bien ubicado...,['https://img4.idealista.com/blur/WEB_DETAIL-X...,...,Chamberí,1.0,1.0,339.756321,True,2,False,2.695,parquet,eclectic
1,Calle de Zurbano,Walter Haus Madrid,4,6,5.0,40.433373,-3.693010,355,"Este exclusivo inmueble, ubicado en el prestig...",['https://img4.idealista.com/blur/WEB_DETAIL-X...,...,Chamberí,1.0,1.0,372.735657,True,1,False,2.720,parquet,eclectic
2,Calle de Hilarión Eslava,Walter Haus Madrid,5,5,2.0,40.435637,-3.716426,265,"Presentamos una espectacular vivienda de lujo,...",['https://img4.idealista.com/blur/WEB_DETAIL-X...,...,Chamberí,1.0,1.0,325.628226,True,4,False,2.745,parquet,coastal
3,Calle de Francisco de Rojas,Walter Haus Madrid,3,3,3.0,40.430204,-3.698828,227,Vivienda totalmente reformada y amueblada que ...,['https://img4.idealista.com/blur/WEB_DETAIL-X...,...,Chamberí,1.0,1.0,371.309203,True,2,False,2.705,parquet,craftsman
4,Paseo Walkway de San Francisco de Sales,DIZA Consultores Inmobiliaria - Propiedades de...,3,2,9.0,40.440927,-3.717716,133,DIZA Consultores presenta atico triplex en el ...,['https://img4.idealista.com/blur/WEB_DETAIL-X...,...,Chamberí,1.0,1.0,499.021336,True,3,False,2.720,sin_predicción,coastal


In [ ]:
import pandas as pd
from PIL import Image
import torch
import os
from transformers import CLIPProcessor, CLIPModel
from tqdm import tqdm # Para ver el progreso

def extract_clip_embeddings(df, image_folder_path, num_images_per_property=1):
    """
    Extrae embeddings de imágenes utilizando el modelo CLIP y los añade al DataFrame.

    Args:
        df (pd.DataFrame): DataFrame con la información de las propiedades.
        image_folder_path (str): Ruta a la carpeta que contiene las imágenes.
        num_images_per_property (int): Número de imágenes a procesar por propiedad.
                                        Por defecto, se procesa solo la primera imagen.

    Returns:
        pd.DataFrame: El DataFrame original con las nuevas columnas de embeddings.
    """
    # Cargar el modelo CLIP y el procesador una sola vez
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

    # Mover el modelo a la GPU si está disponible
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    # Lista para almacenar los embeddings de todas las propiedades
    all_embeddings = []

    print(f"Extrayendo embeddings CLIP para {len(df)} propiedades...")

    # Usamos tqdm para tener una barra de progreso
    for idx in tqdm(df.index, desc="Procesando propiedades"):
        property_embeddings = []
        # Obtener las rutas de las imágenes para esta propiedad
        image_files = sorted([
            f for f in os.listdir(image_folder_path)
            if f.startswith(f'foto_{idx}_') and f.endswith('.webp')
        ])

        if not image_files:
            # Si no hay imágenes, añadir un vector de ceros o NaN del tamaño correcto
            # Para CLIP-ViT-B/32, el tamaño del embedding es 512
            property_embeddings.append(torch.zeros(512).tolist()) # Usamos .tolist() para guardar en DataFrame
            print(f"Advertencia: No hay imágenes para propiedad {idx}. Se añadirá un embedding de ceros.")
        else:
            # Procesar hasta `num_images_per_property` imágenes
            for i, img_file in enumerate(image_files[:num_images_per_property]):
                img_path = os.path.join(image_folder_path, img_file)
                try:
                    image = Image.open(img_path).convert("RGB") # Asegurarse de que sea RGB
                    inputs = processor(images=image, return_tensors="pt").to(device)
                    with torch.no_grad():
                        image_features = model.get_image_features(**inputs)
                    # Convertir a numpy array y luego a lista para almacenar en el DataFrame
                    property_embeddings.append(image_features.squeeze().cpu().numpy().tolist())
                except Exception as e:
                    print(f"Error procesando {img_path}: {str(e)}. Se añadirá un embedding de ceros.")
                    property_embeddings.append(torch.zeros(512).tolist()) # Añadir ceros en caso de error

        # Si se procesan varias imágenes por propiedad, podrías promediar los embeddings
        # o concatenarlos, dependiendo de tu estrategia.
        # Por ahora, si num_images_per_property > 1, solo estamos tomando el embedding de la primera.
        # Si num_images_per_property > 1 y quieres combinar, aquí sería el lugar:
        # Por ejemplo, para promediar si hay más de una imagen y quieres un solo embedding por propiedad:
        # if len(property_embeddings) > 1:
        #     all_embeddings.append(torch.tensor(property_embeddings).mean(dim=0).tolist())
        # else:
        all_embeddings.append(property_embeddings[0]) # Solo la primera imagen o el embedding de ceros

    # Crear nuevas columnas para los embeddings
    # Cada elemento del embedding será una columna separada.
    # El tamaño del embedding para CLIP-ViT-B/32 es 512
    embedding_df = pd.DataFrame(all_embeddings, columns=[f'clip_embedding_{i}' for i in range(512)], index=df.index)

    # Concatenar el DataFrame original con los nuevos embeddings
    df_with_embeddings = pd.concat([df, embedding_df], axis=1)

    return df_with_embeddings

In [ ]:
# Asegúrate de que las rutas y el DataFrame estén cargados correctamente
# df = pd.read_csv('/content/drive/MyDrive/CEU SAN PABLO/TFM/datos_imagenes.csv')
# image_folder_path = '/content/drive/MyDrive/CEU SAN PABLO/TFM/fotos'

# Extraer los embeddings de CLIP (usando la primera imagen de cada propiedad por defecto)
df_with_clip_embeddings = extract_clip_embeddings(df, image_folder_path)

# Guardar el DataFrame actualizado con los embeddings
df_with_clip_embeddings.to_csv('datos_con_clip_embeddings.csv', index=False)

print("\nDataFrame con embeddings CLIP guardado en 'datos_con_clip_embeddings.csv'")
print(df_with_clip_embeddings.head())

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

Extrayendo embeddings CLIP para 1161 propiedades...



Procesando propiedades: 100%|██████████| 1161/1161 [09:23<00:00,  2.06it/s]



DataFrame con embeddings CLIP guardado en 'datos_con_clip_embeddings.csv'
                                   address  \
0                         Calle de Zurbano   
1                         Calle de Zurbano   
2                 Calle de Hilarión Eslava   
3              Calle de Francisco de Rojas   
4  Paseo Walkway de San Francisco de Sales   

                                         agency_name  bathroom_count  \
0                                         Ambassador               4   
1                                 Walter Haus Madrid               4   
2                                 Walter Haus Madrid               5   
3                                 Walter Haus Madrid               3   
4  DIZA Consultores Inmobiliaria - Propiedades de...               3   

   bedroom_count  floor   latitude  longitude  lot_size  \
0              5    5.0  40.435694  -3.691642       344   
1              6    5.0  40.433373  -3.693010       355   
2              5    2.0  40.435637  -3